In [3]:
import os
import openai
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
client = openai.OpenAI() 

In [4]:
# Run this
papers = pd.DataFrame({
    "filename": [
        "2405.10313v1.pdf",
        "2401.03428v1.pdf",
        "2401.09395v2.pdf",
        "2401.13142v3.pdf",
        "2403.02164v2.pdf",
        "2403.12107v1.pdf",
        "2404.10731v1.pdf",
        "2312.11562v5.pdf",
        "2311.02462v2.pdf",
        "2310.15274v1.pdf"
    ],
    "title": [
        "How Far Are We From AGI?",
        "EXPLORING LARGE LANGUAGE MODEL BASED INTELLIGENT AGENTS: DEFINITIONS, METHODS, AND PROSPECTS",
        "CAUGHT IN THE QUICKSAND OF REASONING, FAR FROM AGI SUMMIT: Evaluating LLMs’ Mathematical and Coding Competency through Ontology-guided Interventions",
        "Unsocial Intelligence: an Investigation of the Assumptions of AGI Discourse",
        "Cognition is All You Need The Next Layer of AI Above Large Language Models",
        "Scenarios for the Transition to AGI",
        "What is Meant by AGI? On the Definition of Artificial General Intelligence",
        "A Survey of Reasoning with Foundation Models",
        "Levels of AGI: Operationalizing Progress on the Path to AGI",
        "Systematic AI Approach for AGI: Addressing Alignment, Energy, and AGI Grand Challenges"
    ]
})
papers["filename"] = "papers/" + papers["filename"]
papers

,filename,title
0,papers/2405.10313v1.pdf,How Far Are We From AGI?
1,papers/2401.03428v1.pdf,EXPLORING LARGE LANGUAGE MODEL BASED INTELLIGE...
2,papers/2401.09395v2.pdf,"CAUGHT IN THE QUICKSAND OF REASONING, FAR FROM..."
3,papers/2401.13142v3.pdf,Unsocial Intelligence: an Investigation of the...
4,papers/2403.02164v2.pdf,Cognition is All You Need The Next Layer of AI...
5,papers/2403.12107v1.pdf,Scenarios for the Transition to AGI
6,papers/2404.10731v1.pdf,What is Meant by AGI? On the Definition of Art...
7,papers/2312.11562v5.pdf,A Survey of Reasoning with Foundation Models
8,papers/2311.02462v2.pdf,Levels of AGI: Operationalizing Progress on th...
9,papers/2310.15274v1.pdf,Systematic AI Approach for AGI: Addressing Ali...


In [5]:
# Run this
def upload_file_for_assistant(file_path): 
    uploaded_file = client.files.create(
        file=open(file_path, "rb"),
        purpose='assistants'
    )
    return uploaded_file.id

In [6]:
# In papers, select the filename column, 
# then apply upload_file_for_assistant(),
# then convert the result to a list. 
# Assign to uploaded_file_ids.
uploaded_file_ids = papers["filename"] \
    .apply(upload_file_for_assistant) \
    .to_list()

# See the result
uploaded_file_ids

['file-6hSlm5MHpq8nD1gYo9PacwP4',
 'file-zI8hplkFD5hGZ3Huzy97uqNL',
 'file-BKKnw8CndLZO7VZR9ggyDsDx',
 'file-PnLxYOBh035Ej6I5L45malEL',
 'file-CzHQmM96N6GZLCyNdvkfRyF0',
 'file-SkQ8OuSXsomTiMuETfBUglF9',
 'file-lqX1E8OvmhTcVmzu0KB8r5X2',
 'file-y9VXVZDtYG43JCANx9Ba0yWE',
 'file-7aHeiUdAMDC45PTsT9T2bGhS',
 'file-yZ6Zmsn3JSNy932D9GAktrB1']

In [7]:
# Create a vector store, associating the uploaded file IDs and naming it.
vstore = client.beta.vector_stores.create(
    file_ids = uploaded_file_ids,
    name = "arxiv_agi_papers"
)

# See the result
vstore

VectorStore(id='vs_5tuBMYTkAIqs2jj6XJcWrlH4', created_at=1732048653, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=10, total=10), last_active_at=1732048653, metadata={}, name='arxiv_agi_papers', object='vector_store', status='in_progress', usage_bytes=0, expires_after=None, expires_at=None)

In [26]:
# Run this
assistant_prompt = """
You are Aggie, a knowledgeable and articulate AI assistant specializing in artificial general intelligence (AGI). Your primary role is to read and explain the contents of academic journal articles, particularly those available on arXiv in PDF form. Your target audience comprises data scientists who are familiar with AI concepts but may not be experts in AGI.

When explaining the contents of the papers, follow these guidelines:

Introduction: Start with a brief overview of the paper's title, authors, and the main objective or research question addressed.

Abstract Summary: Provide a concise summary of the abstract, highlighting the key points and findings.

Key Sections and Findings: Break down the paper into its main sections (e.g., Introduction, Methods, Results, Discussion). For each section, provide a summary that includes:

The main points and arguments presented.
Any important methods or techniques used.
Key results and findings.
The significance and implications of these findings.
Conclusion: Summarize the conclusions drawn by the authors, including any limitations they mention and future research directions suggested.

Critical Analysis: Offer a critical analysis of the paper, discussing its strengths and weaknesses. Highlight any innovative approaches or significant contributions to the field of AGI.

Contextual Understanding: Place the paper in the context of the broader field of AGI research. Mention how it relates to other work in the area and its potential impact on future research and applications.

Practical Takeaways: Provide practical takeaways or insights that data scientists can apply in their work. This could include novel methodologies, interesting datasets, or potential areas for collaboration or further study.

Q&A Readiness: Be prepared to answer any follow-up questions that data scientists might have about the paper, providing clear and concise explanations.

Ensure that your explanations are clear, concise, and accessible, avoiding unnecessary jargon. Your goal is to make complex AGI research comprehensible and relevant to data scientists, facilitating their understanding and engagement with the latest advancements in the field.

Limit response length to 30000 TPM.
"""

In [27]:
# Define the assistant. Assign to aggie.
aggie = client.beta.assistants.create(
	name = "Aggie",
	instructions = assistant_prompt,
	model="gpt-4o-mini",
	tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vstore.id]}}
)
    
# See the result
aggie

Assistant(id='asst_J6qHlU2VWY9hmE60xrvCrLqG', created_at=1732049774, description=None, instructions="\nYou are Aggie, a knowledgeable and articulate AI assistant specializing in artificial general intelligence (AGI). Your primary role is to read and explain the contents of academic journal articles, particularly those available on arXiv in PDF form. Your target audience comprises data scientists who are familiar with AI concepts but may not be experts in AGI.\n\nWhen explaining the contents of the papers, follow these guidelines:\n\nIntroduction: Start with a brief overview of the paper's title, authors, and the main objective or research question addressed.\n\nAbstract Summary: Provide a concise summary of the abstract, highlighting the key points and findings.\n\nKey Sections and Findings: Break down the paper into its main sections (e.g., Introduction, Methods, Results, Discussion). For each section, provide a summary that includes:\n\nThe main points and arguments presented.\nAny i

In [28]:
# Create a thread object. Assign to conversation.
conversation = client.beta.threads.create()

# See the result
conversation

Thread(id='thread_fP2UaitWnoWff9furPnzVmnZ', created_at=1732049776, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [29]:
# Add a user message to the conversation. Assign to msg_what_is_agi.
msg_what_is_agi = client.beta.threads.messages.create(
    thread_id=conversation.id,
    role="user",
    content="What are the most common definitions of AGI?"
)

# See the result
msg_what_is_agi

Message(id='msg_6w9KnVA0u8eDDXFMj2pJjejY', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What are the most common definitions of AGI?'), type='text')], created_at=1732049778, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_fP2UaitWnoWff9furPnzVmnZ')

In [31]:
# Run this
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)


In [32]:
# Run this
def run_aggie():
    with client.beta.threads.runs.stream(
        thread_id=conversation.id,
        assistant_id=aggie.id,
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

In [33]:
# Run the assistant
run_aggie()


assistant > file_search


assistant > The term "Artificial General Intelligence" (AGI) encompasses several interpretations and definitions within the AI research community. Here are some of the most common definitions discussed in recent literature:

1. **Human-Level Performance**: Legg and Goertzel defined AGI as a machine capable of performing cognitive tasks that humans typically handle. This definition emphasizes a wide range of cognitive functions but leaves ambiguity regarding the specifics of tasks and the heterogeneity of human intelligence【4:0†source】【4:4†source】.

2. **Broad Learning Abilities**: Shanahan proposed that AGI should be AI that is not restricted to specific tasks but can learn diverse tasks similarly to humans. This definition highlights the importance of metacognitive abilities, like learning and adapting to new situations【4:0†source】【4:4†source】.

3. **Economic Value**: OpenAI defines AGI as "highly autonomous systems that outperform humans at most economicall

In [34]:
# Create another user message, adding it to the conversation. Assign to msg_how_close_is_agi.
msg_how_close_is_agi = client.beta.threads.messages.create(
    thread_id=conversation.id,
    role="user",
    content="How close are we to developing AGI?"
)

# See the result
msg_how_close_is_agi

Message(id='msg_d3OIxUlWALNNAf1cPHRO8eyv', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How close are we to developing AGI?'), type='text')], created_at=1732049801, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_fP2UaitWnoWff9furPnzVmnZ')

In [35]:
# Run the assistant
run_aggie()


assistant > file_search


assistant > Determining how close we are to achieving Artificial General Intelligence (AGI) is a complex and debated topic among researchers. Recent discussions and surveys provide insights into the prevailing perspectives within the AI community:

### Current Sentiments on AGI Development Timelines
1. **Diverse Opinions**: A workshop titled "How Far Are We From AGI," held in early 2024, gathered responses from 138 researchers about their views on AGI timelines. The results revealed a substantial range of opinions:
   - **1-2 years**: 3.6% of respondents believe AGI will be achieved this soon.
   - **2-5 years**: 13.0%.
   - **5-10 years**: 22.5%.
   - **10-20 years**: 23.9%.
   - **20+ years**: A significant 37.0% foresee a more extended timeline【8:0†source】【8:1†source】.

2. **Factors Influencing Opinions**: The differences in timelines suggest that researchers are considering varying bottlenecks and progress indicators. Those optimistically predicting near-